conda install numpy  
conda install pandas  
conda install matplotlib  
conda install ffmpeg  
pip install pydub  
conda install -c conda-forge librosa  
pip install SpeechRecognition  
pip install -U scikit-learn  
conda install -c anaconda nltk

In [1]:
import os
import matplotlib.pyplot as plt

import pydub
pydub.AudioSegment.converter = r"C:/Users/Kinza/anaconda3/envs/pie/Library/bin/ffmpeg.exe" #CHANGE THIS

# Split on questions

In [ ]:
def split_pauses(filename) :
    audio = pydub.AudioSegment.from_file('videos/'+filename+'.mp4','mp4')
    
    audio_chunks = pydub.silence.split_on_silence(audio, 
        # must be silent for at least 1.5 second
        min_silence_len=1500,

        # consider it silent if quieter than -16 dBFS
        silence_thresh=-50
    )

    print('There are',len(audio_chunks),'chunks')
    for i, chunk in enumerate(audio_chunks):
        out_file = 'splits/'+filename+"_chunk{0}.wav".format(i)
        print("Exporting", out_file)
        chunk.export(out_file, format="wav")

In [ ]:
filenames = os.listdir('videos/')
filenames = list(map(lambda s: s.split('.mp4',2)[0],filenames))

# list(map(split_pauses,filenames))

# Lexical feature extraction

In [72]:
import speech_recognition as sr 

def speech_recognition(filename):
    r = sr.Recognizer()
    audio = sr.AudioFile('splits/'+filename+'.wav')
    with audio as source:
        audio_file = r.record(source)
    
    result = r.recognize_google(audio_file,language = 'fr-FR', show_all=False)

    return result

result = speech_recognition(filenames[1])

"en ce qui concerne une expérience dans un milieu international je voulais vous parler de mon stage en États-Unis à l'université de Vienne donc je suis parti avec mes camarades de la promotion de français université américaine nos de 8h était espagnol et nous étions au travail on aussi avec un étudiant dans le car donc c'était vraiment très international donc c'était sur la production électrique toutes les semaines et surtout pouvoir travailler sous pression quand c'est nécessaire"

In [76]:
from nltk.stem.snowball import FrenchStemmer
from nltk import wordpunct_tokenize          
from nltk.corpus import stopwords
from nltk.corpus import words
from string import punctuation
from sklearn.feature_extraction.text import CountVectorizer

In [82]:
class FrenchStemTokenizer(object):
    def __init__(self, remove_non_words=True):
        self.st = FrenchStemmer()
        self.stopwords = set(stopwords.words('french'))
        self.words = set(words.words())
        self.remove_non_words = remove_non_words
    def __call__(self, doc):
        # tokenize words and punctuation
        word_list = wordpunct_tokenize(doc)
        # remove stopwords
        word_list = [word for word in word_list if word not in self.stopwords]
        # remove non words
        if(self.remove_non_words):
            word_list = [word for word in word_list if word in self.words]
        # remove 1-character words
        word_list = [word for word in word_list if len(word)>1]
        # remove non alpha
        word_list = [word for word in word_list if word.isalpha()]
        return [self.st.stem(t) for t in word_list]

In [87]:
countvect = CountVectorizer(tokenizer=FrenchStemTokenizer(remove_non_words=True))
text_fts = countvect.fit_transform([result])
feat2word = {v: k for k, v in countvect.vocabulary_.items()}

countvect.get_feature_names()

['car',
 'international',
 'milieu',
 'pression',
 'product',
 'promot',
 'stag',
 'surtout',
 'travail']